In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import glob
import os
from configuration import *
import numpy as np
debug=1

def balanced_split(p,y,split=0.3):
    p1=[]
    p2=[]
    y1=[]
    y2=[]
    p11=[]
    p22=[]
    y11=[]
    y22=[]
    for i in range(len(y)):
        if y[i]==1:
            p1.append(p[i])
            y1.append(y[i])
        else:
            p2.append(p[i])
            y2.append(y[i])
    pos_size=len(p1)
    neg_size=len(p2)
    if pos_size>neg_size:
        for i in range(neg_size):
            p11.append(p1[i])
            p22.append(p2[i])
            y11.append(y1[i])
            y22.append(y2[i])
    else:
        for i in range(pos_size):
            p22.append(p2[i])
            p11.append(p1[i])
            y11.append(y1[i])
            y22.append(y2[i])
    X=[]
    Y=[]
    X=p11+p22
    Y=y11+y22
    if(debug==1):
        print("Length of input Array=",len(X))
    t1=0
    t2=0
    flag_test=1
    while flag_test==1:
        x,y,x_,y_=train_test_split(X,Y,test_size=split)
        one=0
        zero=0
        for i in range(len(x_)):
            if x_[i]==1:
                one+=1
            else:
                zero+=1
        t1=abs(one-zero)
        one=0
        zero=0
        for i in range(len(y_)):
            if x_[i]==1:
                one+=1
            else:
                zero+=1
        t2=abs(one-zero)
        if debug==1:
            print(t1,t2)
        if t1<=1:
            if t2<=1:
                flag_test=0
                print("done")
    return x,y,x_,y_

def train_rdf():
    k=0
    fds = []
    labels = []
    # Load the positive features
    for feat_path in glob.glob(os.path.join(train_pos_feat_ph,"*.feat")):
        k+=1
        print("+",end="")
        fd = joblib.load(feat_path)
        if(len(fd)==3780):
            fds.append(fd)
            labels.append(1)
        else:
            print("Error in pos",end="")

    # Load the negative features
    for feat_path in glob.glob(os.path.join(train_neg_feat_ph,"*.feat")):
        k+=1
        print("-",end="")
        fd = joblib.load(feat_path)
        if(len(fd)==3780):
            fds.append(fd)
            labels.append(0)
        else:
            print("Error in neg",end="")
    fds=np.array(fds)
    labels=np.array(labels)
    fds_train,fds_test,labels_train,labels_test=balanced_split(fds,labels,split=0.4)
    clf = RandomForestClassifier(n_estimators=100, max_depth=15)
    print("Training a Random Forest Classifier")
    clf.fit(fds_train,labels_train)
    print('Test Accuracy of RDF = ', round(clf.score(fds_test, labels_test), 4))
    if not os.path.isdir(model_path):
        os.makedirs(model_path)
    fd_name="rdf"
    fd_path = os.path.join(model_path, fd_name)
    joblib.dump(clf, fd_path)
    print("Classifier saved to",model_path)

train_rdf()


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++